# Package installation

In [1]:
!pip install openai PyMuPDF
!pip install nltk
!pip install faiss-cpu
!pip install faiss
!pip install llama-cpp-python
!pip install huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 66.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 14.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.14-cp311-cp311-linux_x86_64.whl size=4237783 sha256=de1b54974f28f8728d23c06f4c2e0f2b8b4591677f0e6183155e8e6a9b233b17
  Stored in directory: /root/.cache/pip/wheels/3f/b6/cf/7315ec7b0149210d2d4447d9c3338b36d10e56a1ecddcd35c0
Successfully built llama-cpp-python


## Downlaoding the LLM Models

In [2]:
!huggingface-cli download bartowski/Meta-Llama-3.1-8B-Instruct-GGUF Meta-Llama-3.1-8B-Instruct-Q8_0.gguf --local-dir models --local-dir-use-symlinks False
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q8_0.gguf --local-dir models --local-dir-use-symlinks False
!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q8_0.gguf --local-dir models --local-dir-use-symlinks False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Meta-Llama-3.1-8B-Instruct-Q8_0.gguf: 100% 8.54G/8.54G [08:48<00:00, 16.1MB/s]
Download complete. Moving file to models/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
models/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
mistral-7b-instruct-v0.2.Q8_0.gguf: 100% 7.70G/7.70G [01:20<00:00, 95.5MB/s]
Download complete. Moving file to models/mistral-7b-instruct-v0.2.Q8_0.gguf
models/mistral-7b-instruct-v0.2.Q8_0.gguf
/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use 

# Package loading

In [ ]:
import nltk
import fitz  # PyMuPDF
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from openai import OpenAI
# pretrained embedding model
from sentence_transformers import SentenceTransformer
import faiss, numpy as np
import os
# llama integrations
from llama_cpp import Llama
import json
import pickle
# term frequency per page
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# set the api key and creating the gpt client
api_key = "<put your chatgpt api here>"
client = OpenAI(api_key=api_key)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Loading the data

In [ ]:
# set here the path to where you are storing your files

# --------- To run on Colab --------------
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/<put the path to your files here>'

# --------- To run on locally --------------
#root_path = '<Replace this with the folder where you have your files>'

# add to this list, the name of the files that you want to consider
files_list = ["file_1.pdf", "file_2.pdf"] # an example
#files_list=[ "<Fill this list with the name of the files separated by ,"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load content function

Loading the file contents and creating the metadata.

In [7]:
# this function given the file_path to the file
# will return the text/content of the file
def create_corpus(file_path):

  # extract the file name from the path
  file_name = file_path.split('/')[-1]

  # keeping two lists, one for the content and the other one
  # for the metadata regarding that content
  pages, meta_data = [], []

  # open the document
  doc = fitz.open(file_path)

  # go over each page in the document
  for page_num, page in enumerate(doc):

    # read the content of the whole page
    page_text = page.get_text()

    # add the content of this page to the list
    pages.append(page_text)

    # add the metadata for this page to the list
    meta_data.append({"document": file_name , "page_num": page_num})

  # return the list of pages contents and the metadata
  # related to each page
  return pages, meta_data

# storing the documents and the metadata
documents, metadata = [], []

# loop over the list of files and add them to the lists
for file_path in files_list:
  document_pages, document_metadata = create_corpus(root_path + file_path)
  documents += document_pages
  metadata += document_metadata

# Creating tf-idf and embeddings

At this point given the pages content we will create two presentations

1. tf-idf which is the term frequency and existence per page
2. creating embedding for each page using a pre-trained model

In [8]:
# set these objects to be none first
tfidf_obj, documents_tfidf, embedding_model, page_embeddings = None, None, None, None

# a convention for the file name, 5 initials_ of each file_name
pre_load_prefix = f'{root_path}pre_loading/{"_".join(map(lambda x: x.replace(" ", "")[:5], sorted(files_list)))}'

# check if the we have the files for this combo
if os.path.isfile(f'{pre_load_prefix}_tfidf_obj.pkl'):
    # load the objects and the embeddings
    tfidf_obj = pickle.load(open(f'{pre_load_prefix}_tfidf_obj.pkl', 'rb'))
    documents_tfidf = pickle.load(open(f'{pre_load_prefix}_documents_tfidf.pkl', 'rb'))
    embedding_model = pickle.load(open(f'{pre_load_prefix}_embedding_model.pkl', 'rb'))
    page_embeddings = pickle.load(open(f'{pre_load_prefix}_page_embeddings.pkl', 'rb'))

else:

  # if the directory doesn't exist, create it first
  if not os.path.isdir(f'{root_path}pre_loading'):
    os.makedirs(f'{root_path}pre_loading')

  # creating the tfidf_obj
  tfidf_obj = TfidfVectorizer()
  # create the tfidf vectors
  documents_tfidf = tfidf_obj.fit_transform(documents)
  # a pretrained model to create page embeddings based on the tokens
  # appeared in the pages
  embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", trust_remote_code=True)
  # get the embeddings from the model given the pages content
  page_embeddings  = embedding_model.encode(documents, convert_to_numpy=True, normalize_embeddings=True)

  # save them for later use
  pickle.dump(tfidf_obj, open(f'{pre_load_prefix}_tfidf_obj.pkl', 'wb'))
  pickle.dump(documents_tfidf, open(f'{pre_load_prefix}_documents_tfidf.pkl', 'wb'))
  pickle.dump(embedding_model, open(f'{pre_load_prefix}_embedding_model.pkl', 'wb'))
  pickle.dump(page_embeddings, open(f'{pre_load_prefix}_page_embeddings.pkl', 'wb'))

## FAISS indexing

creating the indexes based on the page embeddings for matching to the query

In [9]:
# creating the indexes based on the page embeddings
# for the comparison
index = faiss.IndexFlatIP(page_embeddings.shape[1])
index.add(page_embeddings)

## Search function + relevant retrieval

This function given the query and the objects and information necessary will return the pages that were relevant to the query

In [10]:
# given the query and additional information, will return the pages tha tare relevant to the query
def hybrid_search_top_k(query, pages_metadata, tfidf_mat, tfidf_obj, page_embeddings, embedding_model, faiss_index, alpha=0.5, k=10):

    # create the tfidf for the query - lexical part
    q_bow = tfidf_obj.transform([query])

    # create the embeddings for the query - semantic part
    q_emb = embedding_model.encode([query], normalize_embeddings=True).astype('float32')

    # find the top k*5 most related documents lexically
    bow_scores = (q_bow @ tfidf_mat.T).toarray().ravel()
    bow_idx    = bow_scores.argsort()[::-1][:k * 5]

    # find the top k*5 top most related documents semantically
    _, dense_idx = faiss_index.search(q_emb, k * 5)
    dense_idx = dense_idx[0]

    # taking the candidates proposed by these 2 approaches
    candidates = np.unique(np.concatenate([dense_idx, bow_idx]))

    # compute the final score for the proposed candidates, a weightening approach
    # to prefer semantic or lexical retrieval
    dense_scores = (page_embeddings[candidates] @ q_emb.T).ravel()     # cosine/IP
    scores = alpha * bow_scores[candidates] + (1 - alpha) * dense_scores

    # find the top k most relevant pages to the query
    order = scores.argsort()[::-1][:k]
    top_ids = candidates[order] # get their page number
    top_scores = scores[order] # get their scores

    # return the socre and the pages that are relevant to the query
    results = [pages_metadata[i] | {"score": float(s)} | {"page_idx" : int(i)} for i, s in zip(top_ids, top_scores)]
    return results

def hybrid_search_top_percentile(query, pages_metadata, tfidf_mat, tfidf_obj, page_embeddings, embedding_model, faiss_index, alpha=0.5, percentile=90.0):
    # create the tfidf for the query - lexical part
    q_bow = tfidf_obj.transform([query])
    bow_scores = (q_bow @ tfidf_mat.T).toarray().ravel()

    # create the embeddings for the query - semantic part
    q_emb = embedding_model.encode([query], normalize_embeddings=True).astype('float32')
    dense_scores = (page_embeddings @ q_emb.T).ravel()  # cosine/IP

    # combine the scores
    combined_scores = alpha * bow_scores + (1 - alpha) * dense_scores

    # find the threholds such that top p precentile be filtered
    thresh = np.percentile(combined_scores, 100 - percentile)

    # select the pages with score higher than the threshold
    selected_idx = np.where(combined_scores >= thresh)[0]

    # select the indexes based on the scores
    sorted_idx = selected_idx[np.argsort(combined_scores[selected_idx])[::-1]]

    # return the results
    results = [{**pages_metadata[i], "combined_score": float(combined_scores[i]), "page_idx": int(i)}for i in sorted_idx]
    return results

# this function given the list of pages content
# and the most relevant contents details will return
# the text of all of those pages
def relevant_pages_content(pages_content, relevant_pages):

  # put together the text of the most relevant queris
  relevant_pages_text = """ """

  # go over each page and put together the content of the page
  for relevant_page in relevant_pages:

    # get the page number
    page_idx = relevant_page['page_idx']

    # get the text of that page
    page_text = pages_content[page_idx]

    # add the text to the previous content of the text
    relevant_pages_text += page_text + '\n'

    return relevant_pages_text

# this function given a text will remove the stopwords from the text in order to
# clean the text and reduce the size of it
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    #filtered_text = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# ChatGpt functions

These functions given the text or the file, will get the summary of the text

In [11]:
# this functin will send the text to the chatgpt and get the summary of the document
# in a structured manner
def summarize_text(text, query):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": "You are a document summarization assistant."},
            {"role": "user", "content": f"""Write me the summary of the following document and point out the key points to fully respond to this query "{query}".

              Return the output in the following structured JSON format:
                {{"summary": "<brief summary>",
                  "key_points": ["<point 1>", "<point 2>", "..."],
                  "section_titles": ["<optional section titles if available>"]}}

              Here is the document:
              {text}
             """}
        ],
        temperature=0.2,
    )
    return response.choices[0].message.content

# this function will analyze the answer given by the llm and
# assign score to them
def score_response_with_chatgpt(user_query, context_text, llm_response):

    system_prompt = "You are an expert evaluator."
    user_prompt = f"""
    Question: {user_query}

    Context: {context_text}

    Response: {llm_response}

    Please evaluate the response on the following criteria, each from 0 to 10:
    1. Relevance — Does it answer the question?
    2. Accuracy — Is it supported by the context?
    3. Clarity — Is it well-structured and easy to understand?

    Also provide an overall score (0–10) and brief comments.

    Return your evaluation in JSON format like this:
    {{
      "relevance": <int>,
      "accuracy": <int>,
      "clarity": <int>,
      "overall_score": <int>,
      "comments": "<text>"
    }}
    Always output valid JSON and nothing else.
    """

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
    )
    content = response.choices[0].message.content
    return json.loads(content)

# LLM Local functions

In [12]:
# this function given the passed text and llm
# will use the model in order to summarize the text
def summarize_text_local(text, llm, query):
    prompt = (f"""Write me the summary of the following document and point out the key points to fully respond to this query "{query}".

              Return the output in the following structured JSON format:
                {{"summary": "<brief summary>\n",
                  "key_points": ["<point 1>", "<point 2>", "...\n"],
                  "section_titles": ["<optional section titles if available>\n"]}}

              Here is the document:
              {text}
             """)

    resp = llm.create_chat_completion(messages=[
        {"role": "system", "content": "You are a document summarization assistant."},
         {"role": "user", "content": prompt}
        ],max_tokens = None, temperature=0.2, response_format={"type":"json_object"})

    return resp['choices'][0]['message']['content']

# LLMs testing

In this section we will set the parameter and the query

In [13]:
# these are going to be used for the serach parts
# alpha in (0, 1): balance between the word matching vs semantic
# higher values gives more importance to word matching
top_k, top_percentile, alpha = 100, 10, 0.6

# set the query of the user
user_query = "When is nitrogen used in any test?"

'''
# if you want to find the top k most relevant pages
relevant_pages = hybrid_search_top_k(query= user_query, pages_metadata=metadata,
                        tfidf_mat=documents_tfidf,tfidf_obj=tfidf_obj,
                        page_embeddings=page_embeddings,faiss_index=index,
                        embedding_model=embedding_model, k = top_k, alpha=0.6)
'''
# if you want to find the top p% highest scored pages
relevant_pages = hybrid_search_top_percentile(query= user_query, pages_metadata=metadata,
                        tfidf_mat=documents_tfidf,tfidf_obj=tfidf_obj,
                        page_embeddings=page_embeddings,faiss_index=index,
                        embedding_model=embedding_model, percentile=top_percentile, alpha=alpha)

# find the most relevant pages
relevant_text = relevant_pages_content(documents, relevant_pages)

## ChatGPT response

In [14]:
# get the response from ChatGPT
gpt_reponse = summarize_text(relevant_text, user_query)

print('Response from ChatGPT:')
print(gpt_reponse, end='\n\n\n')
print("ChatGPT response score: ")
score_response_with_chatgpt(user_query, relevant_text, gpt_reponse)

Response from ChatGPT:
{
  "summary": "The document outlines procedures for strength testing of high-pressure pipelines and facilities, emphasizing the use of water as the primary test medium. However, when water is impractical, nitrogen, air, or natural gas can be used within specified limits. Nitrogen is particularly used when testing at lower stress levels or small volumes, and its required amount can be determined using specific guidelines or software. The document details the importance of selecting appropriate test procedures considering safety, environment, and equipment, and highlights the need for detailed written procedures, especially when back pressure and hydrostatic calculations are involved.",
  "key_points": [
    "Water is the preferred test medium for pressure tests addressing manufacturing and construction defects.",
    "Nitrogen can be used as an alternative test medium when water is impractical, especially at lower stress levels or small volumes.",
    "The amount

{'relevance': 9,
 'accuracy': 10,
 'clarity': 9,
 'overall_score': 9,
 'comments': 'The response effectively addresses the question by explaining when nitrogen is used in testing, supported by specific details from the context. It clearly distinguishes the conditions under which nitrogen is employed and provides relevant procedural information. Minor improvements could include more explicit emphasis on the specific testing scenarios for nitrogen use, but overall, it is comprehensive and well-structured.'}

## Llama response

In [ ]:
%%time

# load the model
llama_Q8 = Llama(model_path="models/llama-2-7b-chat.Q8_0.gguf",  n_ctx=0, verbose=False)

# get the summary of the relevant text
llama_Q8_response = summarize_text_local(relevant_text, llama_Q8, user_query)

print('Response from Llama:')
print(llama_Q8_response, end='\n\n\n')
print("Llama response score: ")
print(score_response_with_chatgpt(user_query, relevant_text, llama_Q8_response))
del llama_Q8 # release the memory

llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


## Meta Llama response

In [ ]:
%%time
Meta_llama = Llama(model_path="models/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", n_ctx=0, verbose=False)

# get the summary of the relevant text
Meta_llama_response = summarize_text_local(relevant_text, Meta_llama, user_query)

print('Response from Meta Llama:')
print(Meta_llama_response, end='\n\n\n')
print("Meta Llama response score: ")
print(score_response_with_chatgpt(user_query, relevant_text, Meta_llama_response))
del Meta_llama # release the memory

llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


Response from Meta Llama:
{"summary": "This document outlines the requirements for hydrostatic testing of high-pressure pipelines and facilities. Prior to starting a hydrotest, several documents and preparations are necessary to ensure safety and compliance with company standards.",
"key_points": ["Verify water used for hydrostatic testing meets GS 167.0232 requirements", "Contact Pipeline Engineering for significant elevation changes", "Verify flanges are torqued properly per GS 180.0020", "Avoid using hoses for filling and dewatering when practicable", "Use rated hoses and fittings for pumps", "Anchor dewatering piping to prevent movement", "Verify employees and contractors are aware of hazards", "Limit access to hydrostatic test dewatering system", "Use sufficient drying pigs/swabs to remove free-standing water", "Dry test segment using an air compressor or other approved methods"],
"section_titles": ["4.6.2 Water used for hydrostatic testing", "4.6.3 Contact Pipeline Engineering", 

## Mistral response


In [ ]:
%%time

# load the llm model
Mistral_llm = Llama(model_path="models/mistral-7b-instruct-v0.2.Q8_0.gguf", n_ctx=0, verbose=False)

# get the summary of the relevant text
Mistral_llm_response = summarize_text_local(relevant_text, Mistral_llm, user_query)

print('Response from Mistral LLM:')
print(Mistral_llm_response, end='\n\n\n')
print("Mistral LLM response score: ")
print(score_response_with_chatgpt(user_query, relevant_text, Mistral_llm_response))
del Mistral_llm # release the memory

llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


Response from Mistral LLM:
{"summary": "This document outlines the requirements for hydrostatic testing of high pressure pipelines and facilities according to Southern California Gas Company's operations standard. Key points include: abiding by the liquid sampling and analysis requirements (GS 167.0232), contacting Pipeline Engineering for significant elevation changes, verifying flange torque, avoiding hose use when possible, using rated hoses and fittings, securing dewatering piping, limiting access to the hydrostatic test dewatering system, using sufficient drying pigs or swabs, and drying the test segment using approved methods.",
"key_points": [
"Abide by the requirements in GS 167.0232 for field sampling and analysis of liquids and solids/sludge",
"Contact Pipeline Engineering for significant elevation changes",
"Verify all flanges have been torqued in proper sequence",
"Avoid hose use when practicable",
"Use rated hoses and fittings for filling and dewatering",
"Securely anchor/